# Imports

In [1]:
import glob
import os
from typing import Dict, List
import warnings
from lightgbm import LGBMClassifier
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import scipy as sp
import scipy.signal
import scipy.stats
from sklearn.model_selection import GroupKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, LeaveOneOut
from sklearn.svm import SVC
from utils import variance_thresholding, standardize, calculate_metrics, calculate_metrics_statistics, DatasetWin, calculate_metrics_from_df
import lightgbm as lgb

In [2]:
# parameters for Welch's method for estimating power spectrum

NPERSEG = 60                    # length of segment
NOVERLAP = int(0.75 * NPERSEG)  # overlap of segments
NFFT = NPERSEG                  # length of FFT
WINDOW = "hann"                 # window function type

# parameters for saving data
PROCESSED_DATA_DIR = "processed_data"
DEPRESJON_PREFIX = "depresjon"
PSYKOSE_PREFIX = "psykose"
HYPERAKTIV_PREFIX = "hyperaktiv"
MAIN_RESULTS_DIR = "results"
DAY_WINDOWS_DIR = "day_windows"
WINDOWS_DIR = "windows"
#DAY_NIGHT_HOURS = (8, 21) # (6, 22) / (8, 21)
DAY_NIGHT_HOURS = [(8, 21), (6, 22)]
#day_night_format = f'{DAY_NIGHT_HOURS[0]}_{DAY_NIGHT_HOURS[1]}' # "6, 22" / "8, 21"

# Manual feature extraction

## Helper functions

In [3]:
def basic_data_cleaning(data: List[List[pd.DataFrame]]) -> List[List[pd.DataFrame]]:
    """
    Assumes DataFrames with "timestamp", "date" and "activity" columns.
    
    Performs cleaning operations:
    - Format "timestamp" to YYYY-MM-DD HH:MM:SS
    - Drop redundant "date" column
    - Convert "activity" to float32
    
    :param data: list of DataFrames
    :returns: list of cleaned DataFrames
    """
    data = [df.copy() for df in data]  # create copy to avoid side effects
    for patient in data:
        for df in patient:
            # Convert and enforce the desired timestamp format
            df["timestamp"] = pd.to_datetime(df["timestamp"], dayfirst=False)
            df["timestamp"] = df["timestamp"].dt.strftime("%Y-%m-%d %H:%M:%S")
            
            # Drop "date" column if it exists
            if "date" in df.columns:
                df.drop("date", axis=1, inplace=True)
            
            # Ensure "activity" column is float32
            df["activity"] = df["activity"].astype(np.float32)
    
    return data


def get_day_part(df: pd.DataFrame, part: str) -> pd.DataFrame:
    """
    For given DataFrame with "timestamp" column returns only those rows that
    correspond to the chosen part of day.
    
    Parts are "day" and "night", defined as:
    - "day": [8:00, 21:00)
    - "night": [21:00, 8:00)
    
    :param df: DataFrame to select rows from
    :param part: part of day, either "day" or "night"
    :param hour_day_start: hour when day start (default 8)
    :param hour_day_end: hour when day ends (starts night) (default 21)
    :returns: DataFrame, subset of rows of df
    """
    if part == "day":
        df = df.loc[(df["timestamp"].dt.hour >= DAY_NIGHT_HOURS[0]) &
                    (df["timestamp"].dt.hour < DAY_NIGHT_HOURS[1])]
    elif part == "night":
        df = df.loc[(df["timestamp"].dt.hour >= DAY_NIGHT_HOURS[1]) |
                    (df["timestamp"].dt.hour < DAY_NIGHT_HOURS[0])]
    else:
        raise ValueError(f'Part should be "day" or "night", got "{part}"')
        
    return df


def fill_missing_activity(df: pd.DataFrame, freq: str = "min") -> pd.DataFrame:
    """
    Fill missing activity values by resampling based on given frequency.
    
    :param df: DataFrame with 'timestamp' and 'activity' columns.
    :param freq: Resampling frequency (default: minute).
    :return: DataFrame with missing values filled.
    """
    df = df.copy() # create copy to avoid side effects
  
    df["timestamp"] = pd.to_datetime(df["timestamp"])

    df.set_index("timestamp", inplace=True)

    # resample to the basic frequency, i.e. minute; this will create NaNs for
    # any rows that may be missing
    df = df.resample(freq).mean()
    
    # recreate index and "timestamp" column
    df.reset_index(inplace=True)

    # fill any NaNs with mean activity value
    df["activity"] = df["activity"].fillna(df["activity"].mean())
    
    return df


def resample(df: pd.DataFrame, freq: str = "H") -> pd.DataFrame:
    """
    Resamples time series DataFrame with given frequency, aggregating each
    segment with a mean.

    :param df: DataFrame with "timestamp" and "activity" columns
    :param freq: resampling frequency passed to Pandas resample() function
    :returns: DataFrame with "timestamp" and "activity" columns
    """
    df = df.copy()  # create copy to avoid side effects
    
    # group with given frequency
    df = df.resample(freq, on="timestamp").mean()

    # recreate "timestamp" column
    df = df.reset_index()

    return df


def proportion_of_zeros(x: np.ndarray) -> float:
    """
    Calculates proportion of zeros in given array, i.e. number of zeros divided
    by length of array.
    
    :param x: 1D Numpy array
    :returns: proportion of zeros
    """
    # we may be dealing with floating numbers, we can't use direct comparison
    zeros_count = np.sum(np.isclose(x, 0))
    return zeros_count / len(x)


def power_spectral_density(df: pd.DataFrame) -> np.ndarray:
    """
    Calculates power spectral density (PSD) from "activity" column of a
    DataFrame.
    
    :param df: DataFrame with "activity" column
    :returns: 1D Numpy array with power spectral density
    """

    activity = df["activity"].values
    nperseg = min(NPERSEG, len(activity))  # Ensure nperseg doesn't exceed data length
    noverlap = int(0.75 * nperseg) 
    
    psd = scipy.signal.welch(
        x=activity,
        fs=(1/60),
        nperseg=nperseg,
        noverlap=noverlap,
        nfft=NFFT,
        window=WINDOW,
        scaling="density"
    )[1]
    return psd


def spectral_flatness(df: pd.DataFrame) -> float:
    """
    Calculates spectral flatness of a signal, i.e. a geometric mean of the
    power spectrum divided by the arithmetic mean of the power spectrum.
    
    If some frequency bins in the power spectrum are close to zero, they are
    removed prior to calculation of spectral flatness to avoid calculation of
    log(0).
    
    :param df: DataFrame with "activity" column
    :returns: spectral flatness value
    """

    activity = df["activity"].values
    nperseg = min(NPERSEG, len(activity))  # Ensure nperseg doesn't exceed data length
    noverlap = int(0.75 * nperseg) 

    power_spectrum = scipy.signal.welch(
        activity,
        fs=(1/60),
        nperseg=nperseg,
        noverlap=noverlap,
        nfft=NFFT,
        window=WINDOW,
        scaling="spectrum"
    )[1]
    
    non_zeros_mask = ~np.isclose(power_spectrum, 0)
    power_spectrum = power_spectrum[non_zeros_mask]
    
    return scipy.stats.gmean(power_spectrum) / power_spectrum.mean()

## Feature extraction

In [4]:
def extract_time_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Extracts features from activity signal in time domain.
    
    :param df_resampled: DataFrame with "activity" column
    :returns: DataFrame with a single row representing features
    """
    X = df["activity"].values
    
    features = {
        "minimum": np.min(X),
        "maximum": np.max(X),
        "mean": np.mean(X),
        "median": np.median(X),
        "variance": np.var(X, ddof=1),  # apply Bessel's correction
        "kurtosis": sp.stats.kurtosis(X),
        "skewness": sp.stats.skew(X),
        "coeff_of_var": sp.stats.variation(X),
        "iqr": sp.stats.iqr(X),
        "trimmed_mean": sp.stats.trim_mean(X, proportiontocut=0.1),
        "entropy": sp.stats.entropy(X, base=2),
        "proportion_of_zeros": proportion_of_zeros(X)
    }
    
    return pd.DataFrame([features])

In [5]:
def extract_frequency_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Extracts features from activity signal in frequency domain, i.e. calculated
    from its Power Spectral Density (PSD).
    
    :param df: DataFrame with "activity" column
    :returns: DataFrame with a single row representing features
    """
    X = power_spectral_density(df)
    
    features = {
        "minimum": np.min(X),
        "maximum": np.max(X),
        "mean": np.mean(X),
        "median": np.median(X),
        "variance": np.var(X),
        "kurtosis": sp.stats.kurtosis(X),
        "skewness": sp.stats.skew(X),
        "coeff_of_var": sp.stats.variation(X),
        "iqr": sp.stats.iqr(X),
        "trimmed_mean": sp.stats.trim_mean(X, proportiontocut=0.1),
        "entropy": sp.stats.entropy(X, base=2),
        "spectral_flatness": spectral_flatness(df)
    }
    
    return pd.DataFrame([features])

In [6]:
def extract_features_for_dataframes(dfs: List[List[pd.DataFrame]], is_condition: bool = True, freq: str = "H") \
        -> Dict[str, pd.DataFrame]:
    """
    Calculates time and frequency features for given DataFrames. Uses given
    frequency for resampling.
    
    Calculates features separately for:
    - full 24hs
    - days: default [8:00, 21:00)
    - nights: default [21:00, 8:00)
    
    :param dfs: list of lists of DataFrames to extract features from; each one has to
    have "timestamp" and "activity" columns
    :param freq: resampling frequency
    :returns: dictionary with keys "full_24h", "day" and "night", corresponding
    to features from given parts of day
    """
    full_dfs = basic_data_cleaning(dfs)
    full_dfs = [[fill_missing_activity(df) for df in patient] for patient in full_dfs]
    full_dfs = [[resample(df, freq=freq) for df in patient] for patient in full_dfs]
    night_dfs = [[get_day_part(df, part="night") for df in patient] for patient in full_dfs]
    day_dfs = [[get_day_part(df, part="day") for df in patient] for patient in full_dfs]

    datasets = {}
    
    
    for part, list_of_dfs in [("full_24h", full_dfs), ("night", night_dfs), ("day", day_dfs)]:
        features = []
        for patient in range(len(list_of_dfs)):
            for day in range(len(list_of_dfs[patient])):
                time_features = extract_time_features(list_of_dfs[patient][day])
                freq_features = extract_frequency_features(list_of_dfs[patient][day])
    
                merged_features = pd.merge(
                    time_features,
                    freq_features,
                    left_index=True,
                    right_index=True,
                    suffixes=("_time", "_freq")
                )
                merged_features['day'] = day + 1
                merged_features['patient_id'] = patient + 1
                if is_condition:
                    merged_features['class'] = 1
                else:
                    merged_features['class'] = 0
                features.append(merged_features)
    
        datasets[part] = pd.concat(features)
        datasets[part].reset_index(drop=True, inplace=True)
    
    return datasets

## Hyperaktiv

In [7]:
path = os.path.join(PROCESSED_DATA_DIR, DAY_WINDOWS_DIR, "hyperaktiv")
dataset = DatasetWin(dirpath=path, sep=',')
condition = dataset.condition
control = dataset.control

In [ ]:
condition_parts_dfs = extract_features_for_dataframes(condition, is_condition=True, freq="H")
control_parts_dfs = extract_features_for_dataframes(control, is_condition=False, freq="H")

datasets = {}

for part in ["full_24h", "night", "day"]:
    condition_df = condition_parts_dfs[part]
    control_df = control_parts_dfs[part]
    max_patient = condition_df['patient_id'].max()
    control_df['patient_id'] += max_patient # changing numeration of patients
    entire_df = pd.concat([condition_df, control_df], ignore_index=True)
    datasets[part] = entire_df

In [219]:
# save windows features
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)
os.makedirs(os.path.join(PROCESSED_DATA_DIR, WINDOWS_DIR), exist_ok=True)
for part, df in datasets.items():
    for hours in DAY_NIGHT_HOURS:
        filename = f"{HYPERAKTIV_PREFIX}_{hours[0]}_{hours[1]}_{part}.csv"
        filepath = os.path.join(PROCESSED_DATA_DIR, WINDOWS_DIR, filename)
        df.to_csv(filepath, index=False, header=True)

In [9]:

for hours in DAY_NIGHT_HOURS:
    filename = f"{HYPERAKTIV_PREFIX}_{hours[0]}_{hours[1]}.csv"
    print(filename)

hyperaktiv_8_21.csv
hyperaktiv_6_22.csv


# Classification

## Classifiers, parameters, constants

In [10]:
classifiers = {
    "GBM": LGBMClassifier(
        objective='binary',
        metric=['auc', 'binary_logloss'],
        force_col_wise=True,
        verbosity=-1
    ),
    "LR": LogisticRegression(
        penalty="elasticnet",
        random_state=0,
        solver="saga",
        max_iter=5000
    ),
    "SVM": SVC(
        kernel="poly",
        cache_size=512,
        max_iter=5000
    ),
    "RF": RandomForestClassifier(
        n_estimators=500,
        criterion="entropy"
    )
}


param_grids = {
    "GBM":{
        'num_leaves': [25, 50, 100, 250, 500],
        'max_depth': [5, 10, 15, 20, 25],
        'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2, 0.25],
        'reg_alpha': [0, 0.1, 0.2, 0.3],
    },
    "LR": {
        "C": [0.001, 0.01, 0.1, 0.5, 1, 2, 5, 10, 25, 50, 100, 500, 1000],
        "class_weight": [None, "balanced"],
        "l1_ratio": [0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5,
                     0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1]
    },
    "SVM": {
        "C": np.logspace(10e-3, 10e3, num=50),
        "gamma": np.logspace(10e-3, 10e3, num=50),
        "class_weight": [None, "balanced"]
    },
    "RF": {
        "class_weight": [None, "balanced", "balanced_subsample"]
    }
}

## Hyperaktiv Classification

In [11]:
dataset = HYPERAKTIV_PREFIX

In [12]:
# read dataset
# create dictionary with data split for night/day/all and also 6/22, 8/21 day/night
datasets = {}
    
for part in ["full_24h", "night", "day"]:
    for hours in DAY_NIGHT_HOURS:
        filename = f"{dataset}_{hours[0]}_{hours[1]}_{part}.csv"
        filepath = os.path.join(PROCESSED_DATA_DIR, WINDOWS_DIR, filename)
        datasets[part] = pd.read_csv(filepath, header=0).dropna()

In [14]:
results_directory = os.path.join(".", MAIN_RESULTS_DIR, DAY_WINDOWS_DIR, "hyperactiv")
predictions_directory = os.path.join('.', MAIN_RESULTS_DIR, DAY_WINDOWS_DIR, "hyperactiv", "predictions")
os.makedirs(results_directory, exist_ok=True)
os.makedirs(predictions_directory, exist_ok=True)

for part in ["day", "night", "full_24h"]:
    for hours in DAY_NIGHT_HOURS:
        part_hours = f"{part}_{hours[0]}_{hours[1]}"
        if part_hours == "full_24h_6_22":
            break
        print(f"PART: {part_hours}")
        X = datasets[part]
        y = datasets[part]['class']
        print(len(X))
        info = X.iloc[:, -3:]
        group_kfold = GroupKFold(n_splits=5)
        fold_num = 0
        all_predictions = pd.DataFrame()
        
        for train_idx, test_idx in group_kfold.split(X, y, groups=X['patient_id']):
            fold_num += 1 
            X = X.iloc[:, :-3]
            print("fold: ", fold_num)
            
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
            X_train, X_test = variance_thresholding(X_train, X_test, threshold=0.005)
            X_train, X_test = standardize(X_train, X_test)

            for clf_type in ["GBM", "LR", "SVM", "RF"]: 
                if clf_type == "GBM":
                    train_data = lgb.Dataset(X_train, label=y_train)
                    test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

                print(f"    {clf_type}")
                
                test_scores = []
                
                model = GridSearchCV(
                    estimator=classifiers[clf_type], 
                    param_grid=param_grids[clf_type], 
                    scoring="accuracy",
                    n_jobs=-1,
                    refit=True,
                    cv=list(GroupKFold(n_splits=5).split(X_train, y_train, info.iloc[train_idx]["patient_id"]))
                )
                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)
                
                metrics = calculate_metrics(model, X_test, y_test)
                # print(metrics)
                test_scores.append(metrics)
        
                # Save individual fold metrics
                pd.DataFrame.from_records(test_scores).to_csv(
                    os.path.join(results_directory, f"test_scores_{hours[0]}_{hours[1]}_{part}_fold_{clf_type}"),
                    index=False
                )

                predictions_dict = {
                    "fold": [fold_num] * len(y_test),
                    'classifier': [clf_type] * len(y_test),
                    'predicted_class': y_pred,
                    'actual_class': y_test,
                    "patient_id": info.iloc[test_idx]["patient_id"].to_list()
                }
                predictions = pd.DataFrame.from_dict(predictions_dict)
                all_predictions = pd.concat([all_predictions, predictions])
        
                # Compute and save final scores for the fold
                final_scores = calculate_metrics_statistics(test_scores)
                df = pd.DataFrame([(key,) + values for key, values in final_scores.items()],
                                columns=['Index', 'Mean', 'Stddev']).set_index('Index')
                df.to_csv(
                    os.path.join(results_directory, f"final_scores_{hours[0]}_{hours[1]}_{part}_{clf_type}"),
                )
            
                for metric, (mean, stddev) in final_scores.items():
                    print(f"      {metric}: {mean:.4f} +- {stddev:.4f}")
                print()
        
        all_predictions.to_csv(
        os.path.join(predictions_directory, f"predictions_{hours[0]}_{hours[1]}_{part}.csv"),
        index=False
        )


PART: day_8_21
547
fold:  1
    GBM
      accuracy: 0.4404 +- 0.0000
      balanced_accuracy: 0.4542 +- 0.0000
      f1: 0.4874 +- 0.0000
      precision: 0.6042 +- 0.0000
      recall: 0.4085 +- 0.0000
      specificity: 0.5000 +- 0.0000
      ROC_AUC: 0.4542 +- 0.0000
      MCC: -0.0879 +- 0.0000

    LR
      accuracy: 0.4128 +- 0.0000
      balanced_accuracy: 0.4392 +- 0.0000
      f1: 0.4386 +- 0.0000
      precision: 0.5814 +- 0.0000
      recall: 0.3521 +- 0.0000
      specificity: 0.5263 +- 0.0000
      ROC_AUC: 0.4392 +- 0.0000
      MCC: -0.1185 +- 0.0000

    SVM
      accuracy: 0.5229 +- 0.0000
      balanced_accuracy: 0.5115 +- 0.0000
      f1: 0.6000 +- 0.0000
      precision: 0.6610 +- 0.0000
      recall: 0.5493 +- 0.0000
      specificity: 0.4737 +- 0.0000
      ROC_AUC: 0.5115 +- 0.0000
      MCC: 0.0220 +- 0.0000

    RF
      accuracy: 0.4220 +- 0.0000
      balanced_accuracy: 0.4707 +- 0.0000
      f1: 0.4112 +- 0.0000
      precision: 0.6111 +- 0.0000
      recall

### Voting

In [16]:
for part in ["day", "night", "full_24h"]:
    for hours in DAY_NIGHT_HOURS:
        part_hours = f"{part}_{hours[0]}_{hours[1]}"
        if part_hours == "full_24h_6_22":
            break
        print(f"PART: {part_hours}")
        predictions = pd.read_csv(os.path.join(predictions_directory, f"predictions_{hours[0]}_{hours[1]}_{part}.csv"))

        grouped = predictions.groupby(['patient_id', 'classifier'])

        most_common_class = (
            grouped['predicted_class']
            .apply(lambda x: x.mode()[0]) 
            .reset_index(name='final_predicted_class')
        )

        final_results = pd.merge(
            most_common_class,
            predictions[['patient_id', 'actual_class']].drop_duplicates(),
            on='patient_id'
        )

        final_results.to_csv(
            os.path.join(predictions_directory, f"final_predictions_{hours[0]}_{hours[1]}_{part}.csv"),
            index=False
        )

        voting_metrics = (
        final_results.groupby('classifier')
        .apply(lambda group: pd.Series(
            calculate_metrics_from_df(group['actual_class'], group['final_predicted_class'])
        ))
        .reset_index()
        )

        display(voting_metrics)
        
        voting_metrics.to_csv(
        os.path.join(predictions_directory, f"voting_scores_{hours[0]}_{hours[1]}_{part}.csv"),
        index=False
        )

PART: day_8_21


,classifier,accuracy,balanced_accuracy,f1,precision,recall,specificity,ROC_AUC,MCC
0,GBM,0.517647,0.519444,0.517647,0.550000,0.488889,0.550,0.519444,0.038889
1,LR,0.411765,0.420833,0.324324,0.413793,0.266667,0.575,0.420833,-0.166692
2,RF,0.470588,0.473611,0.457831,0.500000,0.422222,0.525,0.473611,-0.052984
3,SVM,0.670588,0.658333,0.735849,0.639344,0.866667,0.450,0.658333,0.351130


PART: day_6_22


,classifier,accuracy,balanced_accuracy,f1,precision,recall,specificity,ROC_AUC,MCC
0,GBM,0.458824,0.458333,0.477273,0.488372,0.466667,0.450,0.458333,-0.083195
1,LR,0.423529,0.430556,0.363636,0.437500,0.311111,0.550,0.430556,-0.143084
2,RF,0.505882,0.504167,0.533333,0.533333,0.533333,0.475,0.504167,0.008333
3,SVM,0.541176,0.523611,0.654867,0.544118,0.822222,0.225,0.523611,0.058926


PART: night_8_21


,classifier,accuracy,balanced_accuracy,f1,precision,recall,specificity,ROC_AUC,MCC
0,GBM,0.423529,0.423611,0.436782,0.452381,0.422222,0.425,0.423611,-0.152524
1,LR,0.435294,0.433333,0.466667,0.466667,0.466667,0.400,0.433333,-0.133333
2,RF,0.400000,0.398611,0.426966,0.431818,0.422222,0.375,0.398611,-0.202553
3,SVM,0.470588,0.455556,0.587156,0.500000,0.711111,0.200,0.455556,-0.102869


PART: night_6_22


,classifier,accuracy,balanced_accuracy,f1,precision,recall,specificity,ROC_AUC,MCC
0,GBM,0.494118,0.497222,0.481928,0.526316,0.444444,0.550,0.497222,-0.005577
1,LR,0.470588,0.465278,0.526316,0.500000,0.555556,0.375,0.465278,-0.070430
2,RF,0.458824,0.463889,0.425000,0.485714,0.377778,0.550,0.463889,-0.073247
3,SVM,0.494118,0.477778,0.612613,0.515152,0.755556,0.200,0.477778,-0.053248


PART: full_24h_8_21


,classifier,accuracy,balanced_accuracy,f1,precision,recall,specificity,ROC_AUC,MCC
0,GBM,0.458824,0.455556,0.500000,0.489362,0.511111,0.400,0.455556,-0.089237
1,LR,0.411765,0.415278,0.390244,0.432432,0.355556,0.475,0.415278,-0.170585
2,RF,0.482353,0.480556,0.511111,0.511111,0.511111,0.450,0.480556,-0.038889
3,SVM,0.541176,0.534722,0.597938,0.557692,0.644444,0.425,0.534722,0.071124


## Depresjon

In [17]:
path = os.path.join(PROCESSED_DATA_DIR, DAY_WINDOWS_DIR, "depresjon")
dataset = DatasetWin(dirpath=path, sep=',')
condition = dataset.condition
control = dataset.control

In [221]:
condition_parts_dfs = extract_features_for_dataframes(condition, freq="H")
control_parts_dfs = extract_features_for_dataframes(control, freq="H")

datasets = {}

for part in ["full_24h", "night", "day"]:
    condition_df = condition_parts_dfs[part]
    control_df = control_parts_dfs[part]
    control_df["class"] = 0
    max_patient = condition_df['patient_id'].max()
    control_df['patient_id'] += max_patient # changing numeration of patients
    entire_df = pd.concat([condition_df, control_df], ignore_index=True)
    datasets[part] = entire_df

In [ ]:
for part, df in datasets.items():
    for hours in DAY_NIGHT_HOURS:
        filename = f"{DEPRESJON_PREFIX}_{hours[0]}_{hours[1]}_{part}.csv"
        filepath = os.path.join(PROCESSED_DATA_DIR, WINDOWS_DIR, filename)
        df.to_csv(filepath, index=False, header=True)

## Depresjon classification

In [21]:
dataset = DEPRESJON_PREFIX

In [22]:
# reading
datasets = {}
  
for part in ["full_24h", "night", "day"]:
    for hours in DAY_NIGHT_HOURS:
        filename = f"{dataset}_{hours[0]}_{hours[1]}_{part}.csv"
        filepath = os.path.join(PROCESSED_DATA_DIR, WINDOWS_DIR, filename)
        datasets[part] = pd.read_csv(filepath, header=0).dropna()

In [23]:
from sklearn.model_selection import StratifiedGroupKFold

results_directory = os.path.join(".", MAIN_RESULTS_DIR, DAY_WINDOWS_DIR, "depresjon")
predictions_directory = os.path.join('.', MAIN_RESULTS_DIR, DAY_WINDOWS_DIR, "depresjon", "predictions")
os.makedirs(results_directory, exist_ok=True)
os.makedirs(predictions_directory, exist_ok=True)
predictions = pd.DataFrame(columns=['fold', 'classifier', 'predicted_class', 'actual_class', 'patient_id'])

for part in ["day", "night", "full_24h"]:
    for hours in DAY_NIGHT_HOURS:
        part_hours = f"{part}_{hours[0]}_{hours[1]}"
        if part_hours == "full_24h_6_22":
            break
        print(f"PART: {part_hours}")
        X = datasets[part]
        y = datasets[part]['class']
        print(len(X))
        info = X.iloc[:, -3:]
        group_kfold = StratifiedGroupKFold(n_splits=5)
        fold_num = 0
        all_predictions = pd.DataFrame()
        
        for train_idx, test_idx in group_kfold.split(X, y, groups=X['patient_id']):
            fold_num += 1 
            X = X.iloc[:, :-3]
            print("fold: ", fold_num)
            
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
            X_train, X_test = variance_thresholding(X_train, X_test, threshold=0.005)
            X_train, X_test = standardize(X_train, X_test)
        
            for clf_type in ["GBM", "LR", "SVM", "RF"]: 
                if clf_type == "GBM":
                    train_data = lgb.Dataset(X_train, label=y_train)
                    test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)
                print(f"    {clf_type}")
                
                test_scores = []
                
                model = GridSearchCV(
                    estimator=classifiers[clf_type], 
                    param_grid=param_grids[clf_type], 
                    scoring="accuracy",
                    n_jobs=-1,
                    refit=True,
                    cv=list(StratifiedGroupKFold(n_splits=5).split(X_train, y_train, info.iloc[train_idx]["patient_id"]))
                )
                print(y_train.unique())
                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)
                
                metrics = calculate_metrics(model, X_test, y_test)
                # print(metrics)
                test_scores.append(metrics)
        
                # Save individual fold metrics
                pd.DataFrame.from_records(test_scores).to_csv(
                    os.path.join(results_directory, f"test_scores_{hours[0]}_{hours[1]}_{part}_fold_{clf_type}"),
                    index=False
                )

                predictions_dict = {
                    "fold": [fold_num] * len(y_test),
                    'classifier': [clf_type] * len(y_test),
                    'predicted_class': y_pred,
                    'actual_class': y_test,
                    "patient_id": info.iloc[test_idx]["patient_id"].to_list()
                }
                predictions = pd.DataFrame.from_dict(predictions_dict)
                all_predictions = pd.concat([all_predictions, predictions])
        
                # Compute and save final scores for the fold
                final_scores = calculate_metrics_statistics(test_scores)
                df = pd.DataFrame([(key,) + values for key, values in final_scores.items()],
                                columns=['Index', 'Mean', 'Stddev']).set_index('Index')
                df.to_csv(
                    os.path.join(results_directory, f"final_scores_{hours[0]}_{hours[1]}_{part}_{clf_type}"),
                )
            
                for metric, (mean, stddev) in final_scores.items():
                    print(f"      {metric}: {mean:.4f} +- {stddev:.4f}")
                print()
            
        all_predictions.to_csv(
        os.path.join(predictions_directory, f"predictions_{hours[0]}_{hours[1]}_{part}.csv"),
        index=False
        )

PART: day_8_21
988
fold:  1
    GBM
[1 0]
      accuracy: 0.7320 +- 0.0000
      balanced_accuracy: 0.7134 +- 0.0000
      f1: 0.6438 +- 0.0000
      precision: 0.6528 +- 0.0000
      recall: 0.6351 +- 0.0000
      specificity: 0.7917 +- 0.0000
      ROC_AUC: 0.7134 +- 0.0000
      MCC: 0.4291 +- 0.0000

    LR
[1 0]
      accuracy: 0.7474 +- 0.0000
      balanced_accuracy: 0.7389 +- 0.0000
      f1: 0.6797 +- 0.0000
      precision: 0.6582 +- 0.0000
      recall: 0.7027 +- 0.0000
      specificity: 0.7750 +- 0.0000
      ROC_AUC: 0.7389 +- 0.0000
      MCC: 0.4723 +- 0.0000

    SVM
[1 0]
      accuracy: 0.6443 +- 0.0000
      balanced_accuracy: 0.6944 +- 0.0000
      f1: 0.6601 +- 0.0000
      precision: 0.5194 +- 0.0000
      recall: 0.9054 +- 0.0000
      specificity: 0.4833 +- 0.0000
      ROC_AUC: 0.6944 +- 0.0000
      MCC: 0.4000 +- 0.0000

    RF
[1 0]
      accuracy: 0.7010 +- 0.0000
      balanced_accuracy: 0.6962 +- 0.0000
      f1: 0.6329 +- 0.0000
      precision: 0.5952 

### Voting

In [24]:
for part in ["day", "night", "full_24h"]:
    for hours in DAY_NIGHT_HOURS:
        part_hours = f"{part}_{hours[0]}_{hours[1]}"
        if part_hours == "full_24h_6_22":
            break
        print(f"PART: {part_hours}")
        predictions = pd.read_csv(os.path.join(predictions_directory, f"predictions_{hours[0]}_{hours[1]}_{part}.csv"))

        grouped = predictions.groupby(['patient_id', 'classifier'])

        most_common_class = (
            grouped['predicted_class']
            .apply(lambda x: x.mode()[0]) 
            .reset_index(name='final_predicted_class')
        )

        final_results = pd.merge(
            most_common_class,
            predictions[['patient_id', 'actual_class']].drop_duplicates(),
            on='patient_id'
        )

        final_results.to_csv(
            os.path.join(predictions_directory, f"final_predictions_{hours[0]}_{hours[1]}_{part}.csv"),
            index=False
        )

        voting_metrics = (
        final_results.groupby('classifier')
        .apply(lambda group: pd.Series(
            calculate_metrics_from_df(group['actual_class'], group['final_predicted_class'])
        ))
        .reset_index()
        )

        display(voting_metrics)
        
        voting_metrics.to_csv(
        os.path.join(predictions_directory, f"voting_scores_{hours[0]}_{hours[1]}_{part}.csv"),
        index=False
        )

PART: day_8_21


,classifier,accuracy,balanced_accuracy,f1,precision,recall,specificity,ROC_AUC,MCC
0,GBM,0.709091,0.658288,0.500000,0.888889,0.347826,0.96875,0.658288,0.422101
1,LR,0.745455,0.713995,0.631579,0.800000,0.521739,0.90625,0.713995,0.474019
2,RF,0.690909,0.648777,0.514286,0.750000,0.391304,0.90625,0.648777,0.355370
3,SVM,0.563636,0.618886,0.647059,0.488889,0.956522,0.28125,0.618886,0.304084


PART: day_6_22


,classifier,accuracy,balanced_accuracy,f1,precision,recall,specificity,ROC_AUC,MCC
0,GBM,0.709091,0.658288,0.500000,0.888889,0.347826,0.96875,0.658288,0.422101
1,LR,0.745455,0.713995,0.631579,0.800000,0.521739,0.90625,0.713995,0.474019
2,RF,0.709091,0.658288,0.500000,0.888889,0.347826,0.96875,0.658288,0.422101
3,SVM,0.563636,0.618886,0.647059,0.488889,0.956522,0.28125,0.618886,0.304084


PART: night_8_21


,classifier,accuracy,balanced_accuracy,f1,precision,recall,specificity,ROC_AUC,MCC
0,GBM,0.654545,0.617527,0.486486,0.642857,0.391304,0.84375,0.617527,0.266165
1,LR,0.581818,0.548913,0.410256,0.500000,0.347826,0.75000,0.548913,0.106243
2,RF,0.654545,0.617527,0.486486,0.642857,0.391304,0.84375,0.617527,0.266165
3,SVM,0.490909,0.519701,0.533333,0.432432,0.695652,0.34375,0.519701,0.041421


PART: night_6_22


,classifier,accuracy,balanced_accuracy,f1,precision,recall,specificity,ROC_AUC,MCC
0,GBM,0.654545,0.617527,0.486486,0.642857,0.391304,0.84375,0.617527,0.266165
1,LR,0.581818,0.548913,0.410256,0.500000,0.347826,0.75000,0.548913,0.106243
2,RF,0.654545,0.617527,0.486486,0.642857,0.391304,0.84375,0.617527,0.266165
3,SVM,0.490909,0.519701,0.533333,0.432432,0.695652,0.34375,0.519701,0.041421


PART: full_24h_8_21


,classifier,accuracy,balanced_accuracy,f1,precision,recall,specificity,ROC_AUC,MCC
0,GBM,0.745455,0.707880,0.611111,0.846154,0.478261,0.93750,0.707880,0.482710
1,LR,0.781818,0.757473,0.700000,0.823529,0.608696,0.90625,0.757473,0.549647
2,RF,0.727273,0.698370,0.615385,0.750000,0.521739,0.87500,0.698370,0.430875
3,SVM,0.563636,0.618886,0.647059,0.488889,0.956522,0.28125,0.618886,0.304084


## Psykose

In [25]:
path = os.path.join(PROCESSED_DATA_DIR, DAY_WINDOWS_DIR, "psykose")
dataset = DatasetWin(dirpath=path, sep=',')
condition = dataset.condition
control = dataset.control

In [224]:
condition_parts_dfs = extract_features_for_dataframes(condition, freq="H")
control_parts_dfs = extract_features_for_dataframes(control, freq="H")

datasets = {}

for part in ["full_24h", "night", "day"]:
    condition_df = condition_parts_dfs[part]
    control_df = control_parts_dfs[part]
    control_df["class"] = 0
    max_patient = condition_df['patient_id'].max()
    control_df['patient_id'] += max_patient # changing numeration of patients
    entire_df = pd.concat([condition_df, control_df], ignore_index=True)
    datasets[part] = entire_df

In [ ]:
for part, df in datasets.items():
    for hours in DAY_NIGHT_HOURS:
        filename = f"{PSYKOSE_PREFIX}_{hours[0]}_{hours[1]}_{part}.csv"
        filepath = os.path.join(PROCESSED_DATA_DIR, WINDOWS_DIR, filename)
        df.to_csv(filepath, index=False, header=True)

## Psykose classification

In [26]:
dataset = PSYKOSE_PREFIX

In [27]:
datasets = {}
  
for part in ["full_24h", "night", "day"]:
    for hours in DAY_NIGHT_HOURS:
        filename = f"{dataset}_{hours[0]}_{hours[1]}_{part}.csv"
        filepath = os.path.join(PROCESSED_DATA_DIR, WINDOWS_DIR, filename)
        datasets[part] = pd.read_csv(filepath, header=0).dropna()

In [28]:
results_directory = os.path.join(".", MAIN_RESULTS_DIR, DAY_WINDOWS_DIR, "psykose")
predictions_directory = os.path.join('.', MAIN_RESULTS_DIR, DAY_WINDOWS_DIR, "psykose", "predictions")
os.makedirs(results_directory, exist_ok=True)
os.makedirs(predictions_directory, exist_ok=True)
predictions = pd.DataFrame(columns=['fold', 'classifier', 'predicted_class', 'actual_class', 'patient_id'])

for part in ["day", "night", "full_24h"]:
    for hours in DAY_NIGHT_HOURS:
        part_hours = f"{part}_{hours[0]}_{hours[1]}"
        if part_hours == "full_24h_6_22":
            break
        print(f"PART: {part_hours}")
        X = datasets[part]
        y = datasets[part]['class']
        print(len(X))
        info = X.iloc[:, -3:]
        group_kfold = StratifiedGroupKFold(n_splits=5)
        fold_num = 0
        all_predictions = pd.DataFrame()
        
        for train_idx, test_idx in group_kfold.split(X, y, groups=X['patient_id']):
            fold_num += 1 
            X = X.iloc[:, :-3]
            print("fold: ", fold_num)
            
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
            X_train, X_test = variance_thresholding(X_train, X_test, threshold=0.005)
            X_train, X_test = standardize(X_train, X_test)
        
            for clf_type in ["GBM", "LR", "SVM", "RF"]: 
                if clf_type == "GBM":
                    train_data = lgb.Dataset(X_train, label=y_train)
                    test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)
                print(f"    {clf_type}")
                
                test_scores = []
                
                model = GridSearchCV(
                    estimator=classifiers[clf_type], 
                    param_grid=param_grids[clf_type], 
                    scoring="accuracy",
                    n_jobs=-1,
                    refit=True,
                    cv=list(StratifiedGroupKFold(n_splits=5).split(X_train, y_train, info.iloc[train_idx]["patient_id"]))
                )
                print(y_train.unique())
                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)
                
                metrics = calculate_metrics(model, X_test, y_test)
                # print(metrics)
                test_scores.append(metrics)
        
                # Save individual fold metrics
                pd.DataFrame.from_records(test_scores).to_csv(
                    os.path.join(results_directory, f"test_scores_{hours[0]}_{hours[1]}_{part}_fold_{clf_type}"),
                    index=False
                )

                predictions_dict = {
                    "fold": [fold_num] * len(y_test),
                    'classifier': [clf_type] * len(y_test),
                    'predicted_class': y_pred,
                    'actual_class': y_test,
                    "patient_id": info.iloc[test_idx]["patient_id"].to_list()
                }
                predictions = pd.DataFrame.from_dict(predictions_dict)
                all_predictions = pd.concat([all_predictions, predictions])
        
                # Compute and save final scores for the fold
                final_scores = calculate_metrics_statistics(test_scores)
                df = pd.DataFrame([(key,) + values for key, values in final_scores.items()],
                                columns=['Index', 'Mean', 'Stddev']).set_index('Index')
                df.to_csv(
                    os.path.join(results_directory, f"final_scores_{hours[0]}_{hours[1]}_{part}_{clf_type}"),
                )
            
                for metric, (mean, stddev) in final_scores.items():
                    print(f"      {metric}: {mean:.4f} +- {stddev:.4f}")
                print()
            
        all_predictions.to_csv(
        os.path.join(predictions_directory, f"predictions_{hours[0]}_{hours[1]}_{part}.csv"),
        index=False
        )

PART: day_8_21
971
fold:  1
    GBM
[1 0]
      accuracy: 0.7405 +- 0.0000
      balanced_accuracy: 0.6590 +- 0.0000
      f1: 0.5102 +- 0.0000
      precision: 0.7576 +- 0.0000
      recall: 0.3846 +- 0.0000
      specificity: 0.9333 +- 0.0000
      ROC_AUC: 0.6590 +- 0.0000
      MCC: 0.3965 +- 0.0000

    LR
[1 0]
      accuracy: 0.5892 +- 0.0000
      balanced_accuracy: 0.5071 +- 0.0000
      f1: 0.2830 +- 0.0000
      precision: 0.3659 +- 0.0000
      recall: 0.2308 +- 0.0000
      specificity: 0.7833 +- 0.0000
      ROC_AUC: 0.5071 +- 0.0000
      MCC: 0.0162 +- 0.0000

    SVM
[1 0]
      accuracy: 0.6757 +- 0.0000
      balanced_accuracy: 0.6901 +- 0.0000
      f1: 0.6154 +- 0.0000
      precision: 0.5275 +- 0.0000
      recall: 0.7385 +- 0.0000
      specificity: 0.6417 +- 0.0000
      ROC_AUC: 0.6901 +- 0.0000
      MCC: 0.3630 +- 0.0000

    RF
[1 0]
      accuracy: 0.7459 +- 0.0000
      balanced_accuracy: 0.6702 +- 0.0000
      f1: 0.5347 +- 0.0000
      precision: 0.7500 

### Voting

In [29]:
for part in ["day", "night", "full_24h"]:
    for hours in DAY_NIGHT_HOURS:
        part_hours = f"{part}_{hours[0]}_{hours[1]}"
        if part_hours == "full_24h_6_22":
            break
        print(f"PART: {part_hours}")
        predictions = pd.read_csv(os.path.join(predictions_directory, f"predictions_{hours[0]}_{hours[1]}_{part}.csv"))

        grouped = predictions.groupby(['patient_id', 'classifier'])

        most_common_class = (
            grouped['predicted_class']
            .apply(lambda x: x.mode()[0]) 
            .reset_index(name='final_predicted_class')
        )

        final_results = pd.merge(
            most_common_class,
            predictions[['patient_id', 'actual_class']].drop_duplicates(),
            on='patient_id'
        )

        final_results.to_csv(
            os.path.join(predictions_directory, f"final_predictions_{hours[0]}_{hours[1]}_{part}.csv"),
            index=False
        )

        voting_metrics = (
        final_results.groupby('classifier')
        .apply(lambda group: pd.Series(
            calculate_metrics_from_df(group['actual_class'], group['final_predicted_class'])
        ))
        .reset_index()
        )

        display(voting_metrics)
        
        voting_metrics.to_csv(
        os.path.join(predictions_directory, f"voting_scores_{hours[0]}_{hours[1]}_{part}.csv"),
        index=False
        )

PART: day_8_21


,classifier,accuracy,balanced_accuracy,f1,precision,recall,specificity,ROC_AUC,MCC
0,GBM,0.888889,0.863636,0.842105,1.000000,0.727273,1.00000,0.863636,0.782586
1,LR,0.870370,0.848011,0.820513,0.941176,0.727273,0.96875,0.848011,0.736350
2,RF,0.888889,0.863636,0.842105,1.000000,0.727273,1.00000,0.863636,0.782586
3,SVM,0.592593,0.642045,0.645161,0.500000,0.909091,0.37500,0.642045,0.318529


PART: day_6_22


,classifier,accuracy,balanced_accuracy,f1,precision,recall,specificity,ROC_AUC,MCC
0,GBM,0.888889,0.863636,0.842105,1.000000,0.727273,1.00000,0.863636,0.782586
1,LR,0.870370,0.848011,0.820513,0.941176,0.727273,0.96875,0.848011,0.736350
2,RF,0.888889,0.863636,0.842105,1.000000,0.727273,1.00000,0.863636,0.782586
3,SVM,0.592593,0.642045,0.645161,0.500000,0.909091,0.37500,0.642045,0.318529


PART: night_8_21


,classifier,accuracy,balanced_accuracy,f1,precision,recall,specificity,ROC_AUC,MCC
0,GBM,0.833333,0.823864,0.790698,0.809524,0.772727,0.8750,0.823864,0.652848
1,LR,0.851852,0.846591,0.818182,0.818182,0.818182,0.8750,0.846591,0.693182
2,RF,0.851852,0.846591,0.818182,0.818182,0.818182,0.8750,0.846591,0.693182
3,SVM,0.574074,0.633523,0.646154,0.488372,0.954545,0.3125,0.633523,0.325792


PART: night_6_22


,classifier,accuracy,balanced_accuracy,f1,precision,recall,specificity,ROC_AUC,MCC
0,GBM,0.833333,0.823864,0.790698,0.809524,0.772727,0.8750,0.823864,0.652848
1,LR,0.851852,0.846591,0.818182,0.818182,0.818182,0.8750,0.846591,0.693182
2,RF,0.851852,0.846591,0.818182,0.818182,0.818182,0.8750,0.846591,0.693182
3,SVM,0.574074,0.633523,0.646154,0.488372,0.954545,0.3125,0.633523,0.325792


PART: full_24h_8_21


,classifier,accuracy,balanced_accuracy,f1,precision,recall,specificity,ROC_AUC,MCC
0,GBM,0.907407,0.886364,0.871795,1.000000,0.772727,1.00000,0.886364,0.817499
1,LR,0.814815,0.801136,0.761905,0.800000,0.727273,0.87500,0.801136,0.612809
2,RF,0.888889,0.870739,0.850000,0.944444,0.772727,0.96875,0.870739,0.772853
3,SVM,0.703704,0.714489,0.680000,0.607143,0.772727,0.65625,0.714489,0.421847
